# Project 4: Predict West Nile Virus
### Section 5. Model Exploration

## Problem Statement

1. As an employee of Disease And Treatment Agency, division of Societal Cures In Epidemiology and New Creative Engineering (DATA-SCIENCE), we are tasked to better understand the mosquito population and advise on appropriate interventions which are beneficial and cost-effective for the city.


2. Through this project, we hope to:
- Identify features which are most important to predict presence of West Nile Virus (which can be done by ranking the coefficients of each feature in a logistic regression model)
- Predict the probability of West Nile Virus by location to provide decision makers an effective plan to deploy pesticides throughout the city, which consequently can help to reduce cost.

## Import Libraries

In [3]:
#!pip install shapely
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from shapely import geometry
# from shapely.geometry import Point, Polygon
# import geopandas as gpd
# from datetime import timedelta
# import math

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

## Load Data

In [4]:
# Load datasets
df = pd.read_csv('../data/final_df.csv', index_col='Unnamed: 0')

In [6]:
# Split into train and test (kaggle) data 
train = df[df['dataset']=='train']
test = df[df['dataset']=='test']
print(train.shape)
print(test.shape)

(8304, 227)
(43035, 227)


In [7]:
train.describe()

,latitude,longitude,nummosquitos,tavg,preciptotal,stnpressure,resultdir,avgspeed,is_spray,year_2008,...,codesum_TSRA BR HZ VCTS,codesum_TSRA FG+ BR HZ,codesum_TSRA RA,codesum_TSRA RA BR,codesum_TSRA RA BR HZ,codesum_TSRA RA BR HZ VCTS,codesum_TSRA RA BR VCTS,codesum_TSRA RA VCTS,codesum_VCTS,wnvpresent
count,8304.000000,8304.000000,8304.000000,8304.000000,8294.000000,8304.000000,8304.000000,8304.000000,8304.000000,8304.0,...,8304.000000,8304.0,8304.000000,8304.000000,8304.0,8304.0,8304.000000,8304.0,8304.000000,8304.000000
mean,41.845800,-87.696229,16.095255,72.093931,0.182651,29.262110,17.842245,7.476903,0.008911,0.0,...,0.006142,0.0,0.029383,0.037211,0.0,0.0,0.010597,0.0,0.003974,0.055034
std,0.106658,0.084440,69.585928,7.630330,0.470691,0.118606,9.433945,2.543438,0.093984,0.0,...,0.078132,0.0,0.168889,0.189290,0.0,0.0,0.102402,0.0,0.062918,0.228060
min,41.644612,-87.930995,1.000000,50.000000,0.000000,28.890000,1.000000,2.100000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
25%,41.750498,-87.752411,2.000000,69.000000,0.000000,29.180000,8.000000,5.800000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
50%,41.862292,-87.696269,4.000000,73.000000,0.000000,29.260000,19.000000,7.100000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
75%,41.947227,-87.648064,12.000000,78.000000,0.160000,29.340000,25.000000,9.400000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
max,42.017430,-87.531635,2206.000000,87.000000,3.970000,29.650000,36.000000,16.300000,1.000000,0.0,...,1.000000,0.0,1.000000,1.000000,0.0,0.0,1.000000,0.0,1.000000,1.000000


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8304 entries, 0 to 8303
Columns: 227 entries, latitude to dataset
dtypes: float64(8), int64(218), object(1)
memory usage: 14.4+ MB


## Feature Engineering

In [6]:
# Categorical variables
non_numeric_features = [
    'species',
    'trap',
]

In [7]:
# Create a new dataframe to hold dummies for all categorical features and sale price
dummies_df = train[non_numeric_features]
dummies_df = pd.get_dummies(dummies_df, drop_first=True)
dummy_plus_wnvpresent = pd.concat(objs = [dummies_df, train[['wnvpresent']]] , axis = 1)

In [8]:
# Identify dummy features with high correlation with wnvpresent (correlation >0.4 or <-0.4)
corr_wnvpresent = dummy_plus_wnvpresent.corr().sort_values('wnvpresent', ascending=False)
corr_wnvpresent['wnvpresent']

wnvpresent                1.000000
trap_T900                 0.072840
trap_T003                 0.035874
trap_T225                 0.032690
trap_T086                 0.032537
                            ...   
trap_T043                -0.020760
trap_T148                -0.021435
trap_T017                -0.022883
trap_T046                -0.027178
species_CULEX RESTUANS   -0.098416
Name: wnvpresent, Length: 140, dtype: float64

## Preparing Train-Test (Kaggle) Data and Further Split Train Data into Train and Holdout

In [9]:
# Split train data into X (all features except wnvpresent) and y (wnvpresent)
features = [col for col in train.columns if col != 'wnvpresent']
X = train[features]
y = train['wnvpresent']

In [10]:
y.value_counts(normalize = True)

0.0    0.944966
1.0    0.055034
Name: wnvpresent, dtype: float64

y is highly inbalance, with only about 6% of the data points having West Nile Virus. Hence, it is important to stratify proportionally to ensure that our train and holdout dataset have about the same proportion of presence and absence of West Nile Virus.

In [11]:
# Further split train data into train and holdout data
X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, 
    y,
    stratify = y,
    random_state=42
)

## Model Exploration

### Logistic Regression

In [12]:
# Instantiate model
logreg = LogisticRegression()

# Fit model
logreg.fit(X_train, y_train)

print(f'Logistic Regression Intercept: {logreg.intercept_}')
print(f'Logistic Regression Coefficient: {logreg.coef_}')

ValueError: could not convert string to float: 'train'